# Homework 2: Determinants of Capital Structure

# Applied Corporate Finance – FINA60223A.H2026

# Prof. Jakub Hajda

# Team: Nguyen-Bao Michael Hoang, Philippe Theriault et Nguyen-Tran

## Importation of packages

In [1]:
#Importation of packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

## Data Importation 

In [ ]:
#Load the dataset
